In [2]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
bank=pd.read_csv("/content/Chatbot/BankFAQs.csv")
bank.head()

,Question,Answer,Class
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security


In [4]:
bank["content"] = bank.apply(lambda row: f"Question: {row['Question']}\nAnswer: {row['Answer']}", axis=1)

In [5]:
bank.head()

,Question,Answer,Class,content
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security,Question: Do I need to enter ‘#’ after keying ...
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security,Question: What details are required when I wan...
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security,Question: How should I get the IVR Password i...
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security,Question: How do I register my Mobile number f...
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security,Question: How can I obtain an IVR Password \nA...


In [6]:
!pip install langchain

In [7]:
from langchain.docstore.document import Document

# Prepare documents for LangChain
documents = []
for _, row in bank.iterrows():
    documents.append(Document(page_content=row["content"], metadata={"class": row["Class"]}))

In [8]:
documents[1]

Document(metadata={'class': 'security'}, page_content='Question: What details are required when I want to perform a secure IVR transaction\nAnswer: To perform a secure IVR transaction, you will need your 16-digit Card number, Card expiry date, CVV number, mobile number and IVR password.')

In [9]:
!pip install langchain_community

In [10]:
!pip install sentence-transformers

Loading Data into Chroma DB

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

<ipython-input-11-733749dc74cb>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hg_embeddings = HuggingFaceEmbeddings()
<ipython-input-11-733749dc74cb>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hg_embeddings = HuggingFaceEmbeddings()


In [12]:
!pip install chromadb

In [13]:
from langchain.vectorstores import Chroma

persist_directory = '/content/Chatbot/'

langchain_chroma = Chroma.from_documents(
    documents=documents,
    collection_name="chatbot_finance",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

In [14]:
!pip install bitsandbytes

Loading the Zypher LLM

In [15]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

model_id = 'HuggingFaceH4/zephyr-7b-beta'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cuda:0


In [16]:
!pip install accelerate

In [17]:
!pip install accelerate

In [18]:
!pip install -U bitsandbytes

In [19]:
!pip install -U transformers accelerate
!pip install -U huggingface_hub

In [20]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Building Hugging Face Pipeline to Build LLM Function

In [21]:
# Initialize the query pipeline with increased max_length
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=6000,  # Increase max_length
    max_new_tokens=500,  # Control the number of new tokens generated
    device_map="auto",
)

Device set to use cuda:0


Testing the LLM

In [23]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

llm = HuggingFacePipeline(pipeline=query_pipeline)

question = "What is AI and How it used in Finance Domain?"
response = llm(prompt=question)

full_response =  f"Question: {question}\nAnswer: {response}"
display(Markdown(colorize_text(full_response)))

Both `max_new_tokens` (=500) and `max_length`(=6000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




**<font color='red'>Question:</font>** What is AI and How it used in Finance Domain?


**<font color='green'>Answer:</font>** What is AI and How it used in Finance Domain?

Artificial Intelligence (AI) is a branch of computer science that deals with the creation of intelligent machines that work and react like humans do. AI is a combination of computer science, mathematics, and philosophy. It is a technology that enables machines to learn, reason, and act intelligently.

In the finance domain, AI is used to automate financial processes, analyze financial data, and provide insights and recommendations to financial institutions and individuals. AI is transforming the finance industry by improving efficiency, reducing costs, and enhancing decision-making.

Here are some ways AI is being used in the finance domain:

1. Fraud Detection: AI algorithms can analyze large volumes of financial data in real-time to detect fraudulent activities. AI-powered fraud detection systems can identify patterns and anomalies that are not easily detectable by humans.

2. Risk Management: AI can help financial institutions manage risks by analyzing historical data and predicting future risks. AI-powered risk management systems can provide insights into potential risks and suggest mitigation strategies.

3. Trading and Investment: AI can help financial institutions make informed investment decisions by analyzing market data and predicting future trends. AI-powered trading systems can execute trades automatically based on predefined rules.

4. Customer Experience: AI can help financial institutions provide personalized financial services to their customers by analyzing customer data and providing insights into their financial needs. AI-powered chatbots can provide instant customer support and answer customer queries.

5. Regulatory Compliance: AI can help financial institutions meet regulatory requirements by analyzing regulatory data and providing insights into compliance issues. AI-powered compliance systems can suggest corrective actions and help financial institutions avoid regulatory penalties.

In conclusion, AI is transforming the finance domain by improving efficiency, reducing costs, and enhancing decision-making. AI-powered financial services are becoming increasingly popular as they provide personalized and intelligent financial solutions to individuals and financial institutions. As AI technology continues to evolve, we can expect more innovative and intelligent financial services in the future.